**References**

[Tutorial: Working with Hugging Face Models and Datasets](https://github.com/anyuanay/medium/blob/main/src/working_huggingface/Working_with_HuggingFace_ch4_Fine_Tuning_Pretrained_Model_for_Question_Answering.ipynb)

[Fine-Tuning-for-Question-Answering-SQuAD-IndoQA](https://github.com/PrasetyoWidyantoro/Fine-Tuning-for-Question-Answering-SQuAD-IndoQA/blob/master/fine-tune-squad-dataset.ipynb)

# Import libraries

In [1]:
%pip install evaluate sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from tqdm.auto import tqdm
import evaluate
import numpy as np
import pandas as pd
import os
import json
from sentence_transformers import SentenceTransformer, util

# Global variables

In [3]:
random_seed = 42

kaggle = True

model_name = "racai/distilbert-base-romanian-cased"
model_short_name = "distilbert-romanian"

dataset_name = ("xquad", "xquad.ro")
dataset_short_name = "roitd_aug"

# Import model and tokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/397k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Split the dataset into train, validation and test

In [5]:
ds = load_dataset(dataset_name[0], dataset_name[1])

ds = ds['validation'].train_test_split(test_size=0.2)
ds_train = ds['train']
ds_test = ds['test']

ds_test = ds_test.train_test_split(test_size=0.5)
ds_val = ds_test['train']
ds_test = ds_test['test']

print(len(ds_train), len(ds_val), len(ds_test))

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/244k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1190 [00:00<?, ? examples/s]

952 119 119


In [6]:
with open('/kaggle/input/roitdaug/ds_roitd_augumented.json' if kaggle else 'ds_roitd_augumented.json', 'r') as f:
    ds_roitd = json.load(f) 

ds_train = Dataset.from_pandas(pd.DataFrame(ds_roitd))
ds_train

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 4417
})

# Process the dataset

## Train dataset

In [7]:
def standardize(text: str):
    return text
    # return text.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")

def process_and_tokenize(dataset: Dataset, tokenizer: AutoTokenizer) -> dict:
    # Extract from the dataset and standardize where possible
    questions = [standardize(q).strip() for q in dataset["question"]]
    contexts = [standardize(c) for c in dataset["context"]]
    answers = dataset["answers"]
    
    
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = inputs["overflow_to_sample_mapping"]
    offset_mapping = inputs["offset_mapping"]
    
    start_pos = []
    end_pos = []

    for i, ofs in enumerate(offset_mapping):
        # Get the sample
        sample_i = sample_mapping[i]
        
        # Get the answer for the sample
        answer = answers[sample_i]
        
        # Get the start and end character of the answer
        if len(answer['answer_start']) > 1:
            start = answer["answer_start"][1]
            end = answer["answer_start"][1] + len(answer["text"][1])
        else:
            start = answer["answer_start"][0]
            end = answer["answer_start"][0] + len(answer["text"][0])
        
        # Get the sequence ids
        seq_ids = inputs.sequence_ids(i)

        # Get the start and end token positions
        start_context = seq_ids.index(1)
        end_context = next(j - 1 for j in range(start_context, len(seq_ids)) if seq_ids[j] != 1)

        if ofs[start_context][0] > start or ofs[end_context][1] < end: # If it's impossible
            start_pos.append(0)
            end_pos.append(0)
        else: # Get and append start and end position
            start_pos.append(next((j - 1 for j in range(start_context, end_context + 1) if ofs[j][0] > start), end_context))
            end_pos.append(next((j + 1 for j in range(end_context, start_context - 1, -1) if ofs[j][1] < end), start_context))
            
            # if i < len(dataset['id']) and dataset['id'][i] == '56d6f3500d65d21400198292':
            #     print(start_context, end_context, end_pos[-1], end_pos[-1])

    inputs["start_positions"] = start_pos
    inputs["end_positions"] = end_pos
        
    inputs.pop("overflow_to_sample_mapping")
    inputs.pop("offset_mapping")
    
    return inputs


In [8]:
ds_tok_train = ds_train.map(lambda x: process_and_tokenize(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_train.column_names)
 
print(len(ds_tok_train))

Map:   0%|          | 0/4417 [00:00<?, ? examples/s]

4417


## Validation and test datasets

In [9]:
def process_and_tokenize_val_test(dataset: Dataset, tokenizer: AutoTokenizer) -> dict:
    # Extract from the dataset and standardize where possible
    questions = [standardize(q).strip() for q in dataset["question"]]
    contexts = [standardize(c) for c in dataset["context"]]
    
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = inputs["overflow_to_sample_mapping"]
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_i = sample_mapping[i]
        example_ids.append(dataset["id"][sample_i])

        seq_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        
        for j, ofs in enumerate(offset):
            inputs['offset_mapping'][i][j] = ofs if seq_ids[j] == 1 else None

    inputs["example_id"] = example_ids
    
    inputs.pop("overflow_to_sample_mapping")
    
    return inputs

In [10]:
ds_tok_val = ds_val.map(lambda x: process_and_tokenize_val_test(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_val.column_names)
ds_tok_test = ds_test.map(lambda x: process_and_tokenize_val_test(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_test.column_names)
 
print(len(ds_tok_val), len(ds_tok_test))

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

125 127


# Make metrics

In [11]:
%pip install rouge_score

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=da70d91ce0697ccb53f93e7378f52cb2dc968258c8fc0504170daaca099e45d0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [12]:
squad_metric = evaluate.load("squad")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

In [13]:
def compute_metrics(start_logits: list, end_logits: list, tok: dict, dataset: Dataset, max_answer_length: int = 30, eval_answer: SentenceTransformer = SentenceTransformer("BlackKakapo/stsb-xlm-r-multilingual-ro")) -> dict:
    extracted_features = {}
    
    # Group extracted features by ids
    for i, feature in enumerate(tok):
        if feature["example_id"] not in extracted_features:
            extracted_features[feature["example_id"]] = [i]
        else:
            extracted_features[feature["example_id"]].append(i)
    
    golds_squad = [{"id": data["id"], "answers": data["answers"]} for data in dataset]
    preds_squad = []
    
    golds_bleu = []
    preds_bleu = []
    
    for data in tqdm(dataset):
        answers = []

        # Iterate over all the extracted features
        for i in extracted_features[data["id"]]:
            start_logit = start_logits[i]
            end_logit = end_logits[i]
            offs = tok[i]["offset_mapping"]

            # Get all combinations of start and end positions
            for start_i in range(len(start_logit)):
                for end_i in range(len(end_logit)):
                    # Continue on wrong answers
                    if offs[start_i] is None \
                        or offs[end_i] is None \
                        or end_i < start_i \
                        or end_i - start_i + 1 > max_answer_length:
                        continue
                    
                    # Add text and score
                    answer = {
                        "answer": data["context"][offs[start_i][0] : offs[end_i][1]],
                        "score": start_logit[start_i] + end_logit[end_i]
                    }
                    answers.append(answer)
                    
        preds_squad.append(
            {"id": data['id'], "prediction_text": max(answers, key=lambda x: x["score"])['answer']} 
                if len(answers) > 0 else 
                {"id": data['id'], "prediction_text": ""}
        )
        
        preds_bleu.append(
            max(answers, key=lambda x: x["score"])['answer']
                if len(answers) > 0 else 
                ""
        )
        
        max_i = 0
        max_len = len(data["answers"]["text"][max_i])
        for i in range(1, len(data["answers"]['text'])):
            if len(data["answers"]["text"][i]) > max_len:
                max_len = len(data["answers"]["text"][i])
                max_i = i
        
        golds_bleu.append(data["answers"]['text'][max_i])
        
    return {
        "squad": squad_metric.compute(predictions=preds_squad, references=golds_squad),
        "bleu": bleu_metric.compute(predictions=preds_bleu, references=golds_bleu),
        "rouge": rouge_metric.compute(predictions=preds_bleu, references=golds_bleu),
        "semantic_similarity": util.pytorch_cos_sim(eval_answer.encode(preds_bleu, convert_to_tensor=True), eval_answer.encode(golds_bleu, convert_to_tensor=True)).mean().item() * 100
    }

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Training, evaluation and testing loop for hyperparameters

In [14]:
def output_metrics_to_file(metrics: dict, metric_type: str = None, lr: float = None, epoch: int = None):
    filename = os.path.join("/kaggle/working/" if kaggle else ".", "results", f"{model_short_name}-{dataset_short_name}-type_{metric_type}-lr_{lr}-epoch_{epoch:02}.json" if lr is not None and epoch is not None and metric_type is not None else f"{model_short_name}-{dataset_short_name}.json")
    with open(filename, "w") as f:
        json.dump(metrics, f, indent=4)
        
def save_model(trainer: Trainer, lr: float, epoch: int):
    trainer.save_model(os.path.join("/kaggle/working/" if kaggle else ".", "results", "models", f"{model_short_name}-{dataset_short_name}-{lr}-{epoch}"))
    
def load_model(lr: float, epoch: int) -> AutoModelForQuestionAnswering:
    return AutoModelForQuestionAnswering.from_pretrained(os.path.join("/kaggle/working/" if kaggle else ".", "results", "models", f"{model_short_name}-{dataset_short_name}-{lr}-{epoch}"))

## Get maximum answer length within 2 standard deviations from the mean of the training dataset

In [15]:
# mean_answer_length = np.mean([len(a["text"][0]) for a in ds_train["answers"]])
# std_dev_answer_length = np.std([len(a["text"][0]) for a in ds_train["answers"]])

# two_std_devs_above = mean_answer_length + 2 * std_dev_answer_length

In [16]:
all_models_eval = None

def tet_loop(lr_list: list, epochs: int, batch_size: int) -> None:
    global all_models_eval
    all_models_eval = {"validation": {lr:{epoch:[] for epoch in range(1, epochs + 1)} for lr in lr_list}, "test": {lr:{epoch:[] for epoch in range(1, epochs + 1)} for lr in lr_list}}
    
    eval_answer = SentenceTransformer("BlackKakapo/stsb-xlm-r-multilingual-ro")
    
    for lr in lr_list:
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)

        warmup_steps = int(len(ds_tok_train) / batch_size * 1 / 10)
        args = TrainingArguments(
                output_dir="./results",
                eval_strategy="no",
                save_strategy="epoch",
                learning_rate=lr,
                num_train_epochs=1,
                weight_decay=0.01,
                per_device_train_batch_size=batch_size, 
                report_to="none",
                save_total_limit=1,
                warmup_steps=warmup_steps
            )
        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=ds_tok_train,
            eval_dataset=ds_tok_val,
            tokenizer=tokenizer
        )
        
        for epoch in range(1, epochs + 1):
            print(f"Training with lr={lr} and epoch={epoch}")
            
            trainer.train()
            
            preds = trainer.predict(ds_tok_val)
            start_logits, end_logits = preds[0][0], preds[0][1]
            computed_metrics = compute_metrics(start_logits, end_logits, ds_tok_val, ds_val, 30, eval_answer)
            
            # output_metrics_to_file(computed_metrics, metric_type='validation', lr=lr, epoch=epoch)
            print(f"Validation for lr {lr} epoch {epoch}: ")
            # print(computed_metrics)
            print(computed_metrics['squad'])
            print(computed_metrics['semantic_similarity'])
            
            all_models_eval['validation'][lr][epoch] = computed_metrics
            
            preds = trainer.predict(ds_tok_test)
            start_logits2, end_logits2 = preds[0][0], preds[0][1]
            computed_metrics = compute_metrics(start_logits2, end_logits2, ds_tok_test, ds_test, 30, eval_answer)
            
            
            # output_metrics_to_file(computed_metrics, metric_type='test', lr=lr, epoch=epoch)
            all_models_eval['test'][lr][epoch] = computed_metrics
            

            print(f"Test for lr {lr} epoch {epoch}: ")
            print(computed_metrics['squad'])
            print(computed_metrics['semantic_similarity'])
            
            # save_model(trainer, lr, epoch)
            
        try:
            del trainer
            del model
            # device = cuda.get_current_device()
            # device.reset()
        except Exception as e:
            print(e)
            
            print(f"Finished training, validating and testing with lr={lr} and epochs={epochs}")

In [17]:
tet_loop(lr_list=[1e-4, 1e-3, 1e-5], epochs=8, batch_size=16)

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at racai/distilbert-base-romanian-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training with lr=0.0001 and epoch=1


Step,Training Loss


  0%|          | 0/119 [00:00<?, ?it/s]

...

Validation for lr 1e-05 epoch 8: 
{'exact_match': 3.361344537815126, 'f1': 14.751860576098906}
10.472968965768814


  0%|          | 0/119 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Test for lr 1e-05 epoch 8: 
{'exact_match': 0.0, 'f1': 13.124607559604579}
9.465247392654419


In [18]:
output_metrics_to_file(all_models_eval)

In [19]:
!zip results_ro_aug.zip results/*.json

  adding: results/distilbert-romanian-roitd_aug.json (deflated 84%)


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
